In [26]:
try:
  from google.colab import drive
  drive.mount('/content/drive/')
  shared_drive_foldername = 'NUTRECON'
  root = '/content/drive/Shareddrives/{}/'.format(shared_drive_foldername)
  !pip install pandas==1.4.2
  print('Running Code in Colab')
# ...or locally
except:
  # define the local directory of the project
  root = 'D:/FV/Projects/NUTRECON/nutreconDrive/'
  # root = '/mnt/data1/francisco/Projects/NUTRECON/nutreconDrive/'
  print('Running Code locally')

import numpy as np
import pandas as pd
import sys
sys.path.append(root + 'python')
import nutrecon_simulation as sim
import neuroeconomics_jaimeNLL as neconJaime
import neuroeconomics as necon
from variableCoding import Vars
import seaborn as sns
import pandas as pd
from scipy.optimize import minimize
import warnings
from tqdm import tqdm

_v_ = Vars()

import plotly.express as px

beahviour_cols = necon.optimize_cols[:-4]

Running Code locally


In [27]:
# df, columns = ['ref_prob', 'ref_qt', 'ref_alpha', 
#            'lott_prob', 'lott_qt', 'lott_alpha', 
#            'beta', 'sFactor', 'choice']):
    
st_id = 'same'
mt_id = 'mixed'
money_id = 'Money'
cPlus_id = 'CS+'
cMinus_id = 'CS-'

optimize_cols = ['trial_type',
              'ref_type', 'ref_qt', 'ref_prob',
              'lott_type', 'lott_qt', 'lott_prob',
              'choice',
              'ref_alpha_est', 'lott_alpha_est', 'beta_est',
              'sFactor_est']

def _get_choices(df, columns = ['ref_prob', 'ref_qt', 'ref_alpha', 
           'lott_prob', 'lott_qt', 'lott_alpha', 
           'beta', 'sFactor']):
    
    numeric_df = df.apply(pd.to_numeric, errors='ignore').select_dtypes('float', 'int')

    arr = numeric_df.values
    arr_labels = numeric_df.columns

    ref_prob_arr = arr[:, arr_labels == columns[0]]
    ref_qt_arr = arr[:, arr_labels == columns[1]]
    ref_alpha_arr = arr[:, arr_labels == columns[2]]

    lott_prob_arr = arr[:, arr_labels == columns[3]]
    lott_qt_arr = arr[:, arr_labels == columns[4]]
    lott_alpha_arr = arr[:, arr_labels == columns[5]]

    beta_arr = arr[:, arr_labels == columns[6]]
    sFactor_arr = arr[:, arr_labels == columns[7]]

    euR = necon._calculate_EU(ref_prob_arr, ref_qt_arr, ref_alpha_arr)
    euL = necon._calculate_EU(lott_prob_arr, lott_qt_arr, lott_alpha_arr)
    pL = necon._calculate_pL(euL, euR, beta_arr, sFactor_arr)
    choices = np.random.uniform(size=euR.shape) < pL
    return choices

def _compute_nll(ref_prob_arr, ref_qt_arr, ref_alpha_arr,
                 lott_prob_arr, lott_qt_arr, lott_alpha_arr,
                 beta_arr, sFactor_arr, 
                 choice_arr):
    
    euR = ref_prob_arr*ref_qt_arr**ref_alpha_arr 
    euL = lott_prob_arr*lott_qt_arr**lott_alpha_arr

    # Compute things once, sign flip to possibly save one operation
    y = beta_arr*(euR - euL*sFactor_arr)
    chose_ref = choice_arr==False

    # values to be summed up, already negative:
    nll_v = np.log(1 + np.exp(y))
    nll_v[chose_ref] = nll_v[chose_ref] - y[chose_ref]

    return np.sum(nll_v)


def _get_st_nll(params, df):
    '''
    Computes negative logLikelihood

    columns should have the following order:
              ['trial_type',
              'ref_type', 'ref_qt', 'ref_prob',
              'lott_type', 'lott_qt', 'lott_prob',
              'choice',
              'ref_alpha_est', 'lott_alpha_est', 'beta_est',
              'sFactor_est']
              '''
    
    cols = df.columns
    reff_type_arr = df[cols[1]].values
    ref_prob_arr = df[cols[3]].values
    ref_qt_arr = df[cols[2]].values
    lott_type_arr = df[cols[4]].values
    lott_prob_arr = df[cols[6]].values
    lott_qt_arr = df[cols[5]].values
    choice_arr = df[cols[7]].values

    ref_alpha_arr = np.zeros(lott_type_arr.shape)
    lott_alpha_arr = np.zeros(lott_type_arr.shape)
    beta_arr = np.zeros(lott_type_arr.shape)
    sFactor_arr = np.ones(lott_type_arr.shape)

    # Unpack params
    if len(params) == 4:
       print('hello')
       # Three alphas, one beta 
       (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
       beta) = params
       
       # unpack params for trial in question
       ref_alpha_arr[reff_type_arr == money_id] = st_money_alpha
       ref_alpha_arr[reff_type_arr == cPlus_id] = st_cPlus_alpha
       ref_alpha_arr[reff_type_arr == cMinus_id] = st_cMinus_alpha

       lott_alpha_arr[lott_type_arr == money_id] = st_money_alpha
       lott_alpha_arr[lott_type_arr == cPlus_id] = st_cPlus_alpha
       lott_alpha_arr[lott_type_arr == cMinus_id] = st_cMinus_alpha

       beta_arr[:] = beta

    elif len(params) == 6:
       # Three alphas, three betas 
       (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
       st_money_beta, st_cPlus_beta, st_cMinus_beta) = params

       # unpack params for trial in question
       ref_alpha_arr[reff_type_arr == money_id] = st_money_alpha
       ref_alpha_arr[reff_type_arr == cPlus_id] = st_cPlus_alpha
       ref_alpha_arr[reff_type_arr == cMinus_id] = st_cMinus_alpha

       lott_alpha_arr[lott_type_arr == money_id] = st_money_alpha
       lott_alpha_arr[lott_type_arr == cPlus_id] = st_cPlus_alpha
       lott_alpha_arr[lott_type_arr == cMinus_id] = st_cMinus_alpha
       
       beta_arr[lott_type_arr == money_id] = st_money_beta
       beta_arr[lott_type_arr == cPlus_id] = st_cPlus_beta
       beta_arr[lott_type_arr == cMinus_id] = st_cMinus_beta

    negloglikelihood = _compute_nll(ref_prob_arr, ref_qt_arr, ref_alpha_arr,
                                   lott_prob_arr, lott_qt_arr, lott_alpha_arr,
                                   beta_arr, sFactor_arr, 
                                   choice_arr)

    return negloglikelihood


def _get_mt_nll(params, df):
    '''
    Computes negative logLikelihood

    columns should have the following order:
              ['trial_type',
              'ref_type', 'ref_qt', 'ref_prob',
              'lott_type', 'lott_qt', 'lott_prob',
              'choice',
              'ref_alpha_est', 'lott_alpha_est', 'beta_est',
              'sFactor_est']
              '''
    
    cols = df.columns
    ref_prob_arr = df[cols[3]].values
    ref_qt_arr = df[cols[2]].values
    lott_type_arr = df[cols[4]].values
    lott_prob_arr = df[cols[6]].values
    lott_qt_arr = df[cols[5]].values
    choice_arr = df[cols[7]].values

    ref_alpha_arr = np.zeros(lott_type_arr.shape)
    lott_alpha_arr = np.zeros(lott_type_arr.shape)
    beta_arr = np.zeros(lott_type_arr.shape)
    sFactor_arr = np.ones(lott_type_arr.shape)


    # Unpack params
    if len(params) == 2:
       # two scaling factors
       (cPlus_sFactor, cMinus_sFactor) = params
       
       # unpack params for trial in question
       sFactor_arr[lott_type_arr == cPlus_id] = cPlus_sFactor
       sFactor_arr[lott_type_arr == cMinus_id] = cMinus_sFactor

    elif len(params) == 4:
       # Two betas and two scaling factors
       (mt_cPlus_beta, mt_cMinus_beta,
        cPlus_sFactor, cMinus_sFactor) = params

       # unpack params for trial in question
       sFactor_arr[lott_type_arr == cPlus_id] = cPlus_sFactor
       sFactor_arr[lott_type_arr == cMinus_id] = cMinus_sFactor

       beta_arr[lott_type_arr == cMinus_id] = mt_cPlus_beta
       beta_arr[lott_type_arr == money_id] = mt_cMinus_beta

    negloglikelihood = _compute_nll(ref_prob_arr, ref_qt_arr, ref_alpha_arr,
                                   lott_prob_arr, lott_qt_arr, lott_alpha_arr,
                                   beta_arr, sFactor_arr, 
                                   choice_arr)

    return negloglikelihood

def _get_nll(params, df):
    '''
    Computes negative logLikelihood

    columns should have the following order:
              ['trial_type',
              'ref_type', 'ref_qt', 'ref_prob',
              'lott_type', 'lott_qt', 'lott_prob',
              'choice',
              'ref_alpha_est', 'lott_alpha_est', 'beta_est',
              'sFactor_est']
              '''
    
    cols = df.columns
    trial_type_arr = df[cols[0]].values
    reff_type_arr = df[cols[1]].values
    ref_prob_arr = df[cols[3]].values
    ref_qt_arr = df[cols[2]].values
    lott_type_arr = df[cols[4]].values
    lott_prob_arr = df[cols[6]].values
    lott_qt_arr = df[cols[5]].values
    choice_arr = df[cols[7]].values

    ref_alpha_arr = np.zeros(lott_type_arr.shape)
    lott_alpha_arr = np.zeros(lott_type_arr.shape)
    beta_arr = np.zeros(lott_type_arr.shape)
    sFactor_arr = np.ones(lott_type_arr.shape)

    # Unpack params
    if len(params) == 6:
       # Three alphas, one beta and two scaling factors (beta is unpacked directly)
       (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
       beta, 
       cPlus_sFactor, cMinus_sFactor) = params
       
       # unpack params for trial in question
       ref_alpha_arr[reff_type_arr == money_id] = st_money_alpha
       ref_alpha_arr[reff_type_arr == cPlus_id] = st_cPlus_alpha
       ref_alpha_arr[reff_type_arr == cMinus_id] = st_cMinus_alpha

       lott_alpha_arr[lott_type_arr == money_id] = st_money_alpha
       lott_alpha_arr[lott_type_arr == cPlus_id] = st_cPlus_alpha
       lott_alpha_arr[lott_type_arr == cMinus_id] = st_cMinus_alpha

       beta_arr[:] = beta

       sFactor_arr[lott_type_arr == cPlus_id] = cPlus_sFactor
       sFactor_arr[lott_type_arr == cMinus_id] = cMinus_sFactor

    elif len(params) == 10:
       # Three alphas, five betas, two scaling factors 
       (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
        st_money_beta, st_cPlus_beta, st_cMinus_beta,
        mt_cPlus_beta, mt_cMinus_beta,
        cPlus_sFactor, cMinus_sFactor) = params

       # unpack params for trial in question
       ref_alpha_arr[reff_type_arr == money_id] = st_money_alpha
       ref_alpha_arr[reff_type_arr == cPlus_id] = st_cPlus_alpha      # This should never happen
       ref_alpha_arr[reff_type_arr == cMinus_id] = st_cMinus_alpha    # This should never happen

       lott_alpha_arr[lott_type_arr == money_id] = st_money_alpha
       lott_alpha_arr[lott_type_arr == cPlus_id] = st_cPlus_alpha
       lott_alpha_arr[lott_type_arr == cMinus_id] = st_cMinus_alpha
       
       beta_arr[(trial_type_arr == st_id) & (lott_type_arr == money_id)] = st_money_beta
       beta_arr[(trial_type_arr == st_id) & (lott_type_arr == cPlus_id)] = st_cPlus_beta
       beta_arr[(trial_type_arr == st_id) & (lott_type_arr == cMinus_id)] = st_cMinus_beta
       beta_arr[(trial_type_arr == mt_id) & (lott_type_arr == cMinus_id)] = mt_cPlus_beta
       beta_arr[(trial_type_arr == mt_id) & (lott_type_arr == money_id)] = mt_cMinus_beta

       sFactor_arr[lott_type_arr == cPlus_id] = cPlus_sFactor
       sFactor_arr[lott_type_arr == cMinus_id] = cPlus_sFactor


    negloglikelihood = _compute_nll(ref_prob_arr, ref_qt_arr, ref_alpha_arr,
                                   lott_prob_arr, lott_qt_arr, lott_alpha_arr,
                                   beta_arr, sFactor_arr, 
                                   choice_arr)

    return negloglikelihood



In [28]:
import numpy as np
import random 
import pandas as pd
from scipy.optimize import minimize


st_id = 'same'
mt_id = 'mixed'
money_id = 'Money'
cPlus_id = 'CS+'
cMinus_id = 'CS-'

optimize_cols = ['trial_type',
              'ref_type', 'ref_qt', 'ref_prob',
              'lott_type', 'lott_qt', 'lott_prob',
              'choice',
              'ref_alpha_est', 'lott_alpha_est', 'beta_est',
              'sFactor_est']

def _get_choices(df, columns = ['ref_prob', 'ref_qt', 'ref_alpha', 
           'lott_prob', 'lott_qt', 'lott_alpha', 
           'beta', 'sFactor']):
    
    numeric_df = df.apply(pd.to_numeric, errors='ignore').select_dtypes('float', 'int')

    arr = numeric_df.values
    arr_labels = numeric_df.columns

    ref_prob_arr = arr[:, arr_labels == columns[0]]
    ref_qt_arr = arr[:, arr_labels == columns[1]]
    ref_alpha_arr = arr[:, arr_labels == columns[2]]

    lott_prob_arr = arr[:, arr_labels == columns[3]]
    lott_qt_arr = arr[:, arr_labels == columns[4]]
    lott_alpha_arr = arr[:, arr_labels == columns[5]]

    beta_arr = arr[:, arr_labels == columns[6]]
    sFactor_arr = arr[:, arr_labels == columns[7]]

    euR = necon._calculate_EU(ref_prob_arr, ref_qt_arr, ref_alpha_arr)
    euL = necon._calculate_EU(lott_prob_arr, lott_qt_arr, lott_alpha_arr)
    pL = necon._calculate_pL(euL, euR, beta_arr, sFactor_arr)
    choices = np.random.uniform(size=euR.shape) < pL
    return choices

def _compute_nll(ref_prob_arr, ref_qt_arr, ref_alpha_arr,
                 lott_prob_arr, lott_qt_arr, lott_alpha_arr,
                 beta_arr, sFactor_arr, 
                 choice_arr):
    
    euR = ref_prob_arr*ref_qt_arr**ref_alpha_arr 
    euL = lott_prob_arr*lott_qt_arr**lott_alpha_arr

    # Compute things once, sign flip to possibly save one operation
    y = beta_arr*(euR - euL*sFactor_arr)
    chose_ref = choice_arr==False

    # values to be summed up, already negative:
    nll_v = np.log(1 + np.exp(y))
    nll_v[chose_ref] = nll_v[chose_ref] - y[chose_ref]

    return np.sum(nll_v)


def _get_st_nll(params, df):
    '''
    Computes negative logLikelihood

    columns should have the following order:
              ['trial_type',
              'ref_type', 'ref_qt', 'ref_prob',
              'lott_type', 'lott_qt', 'lott_prob',
              'choice',
              'ref_alpha_est', 'lott_alpha_est', 'beta_est',
              'sFactor_est']
              '''
    
    cols = df.columns
    reff_type_arr = df[cols[1]].values
    ref_prob_arr = df[cols[3]].values
    ref_qt_arr = df[cols[2]].values
    lott_type_arr = df[cols[4]].values
    lott_prob_arr = df[cols[6]].values
    lott_qt_arr = df[cols[5]].values
    choice_arr = df[cols[7]].values

    ref_alpha_arr = np.zeros(lott_type_arr.shape)
    lott_alpha_arr = np.zeros(lott_type_arr.shape)
    beta_arr = np.zeros(lott_type_arr.shape)
    sFactor_arr = np.ones(lott_type_arr.shape)

    # Unpack params
    if len(params) == 4:
       print('hello')
       # Three alphas, one beta 
       (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
       beta) = params
       
       # unpack params for trial in question
       ref_alpha_arr[reff_type_arr == money_id] = st_money_alpha
       ref_alpha_arr[reff_type_arr == cPlus_id] = st_cPlus_alpha
       ref_alpha_arr[reff_type_arr == cMinus_id] = st_cMinus_alpha

       lott_alpha_arr[lott_type_arr == money_id] = st_money_alpha
       lott_alpha_arr[lott_type_arr == cPlus_id] = st_cPlus_alpha
       lott_alpha_arr[lott_type_arr == cMinus_id] = st_cMinus_alpha

       beta_arr[:] = beta

    elif len(params) == 6:
       # Three alphas, three betas 
       (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
       st_money_beta, st_cPlus_beta, st_cMinus_beta) = params

       # unpack params for trial in question
       ref_alpha_arr[reff_type_arr == money_id] = st_money_alpha
       ref_alpha_arr[reff_type_arr == cPlus_id] = st_cPlus_alpha
       ref_alpha_arr[reff_type_arr == cMinus_id] = st_cMinus_alpha

       lott_alpha_arr[lott_type_arr == money_id] = st_money_alpha
       lott_alpha_arr[lott_type_arr == cPlus_id] = st_cPlus_alpha
       lott_alpha_arr[lott_type_arr == cMinus_id] = st_cMinus_alpha
       
       beta_arr[lott_type_arr == money_id] = st_money_beta
       beta_arr[lott_type_arr == cPlus_id] = st_cPlus_beta
       beta_arr[lott_type_arr == cMinus_id] = st_cMinus_beta

    negloglikelihood = _compute_nll(ref_prob_arr, ref_qt_arr, ref_alpha_arr,
                                   lott_prob_arr, lott_qt_arr, lott_alpha_arr,
                                   beta_arr, sFactor_arr, 
                                   choice_arr)

    return negloglikelihood


def _get_mt_nll(params, df):
    '''
    Computes negative logLikelihood

    columns should have the following order:
              ['trial_type',
              'ref_type', 'ref_qt', 'ref_prob',
              'lott_type', 'lott_qt', 'lott_prob',
              'choice',
              'ref_alpha_est', 'lott_alpha_est', 'beta_est',
              'sFactor_est']
              '''
    
    cols = df.columns
    ref_prob_arr = df[cols[3]].values
    ref_qt_arr = df[cols[2]].values
    lott_type_arr = df[cols[4]].values
    lott_prob_arr = df[cols[6]].values
    lott_qt_arr = df[cols[5]].values
    choice_arr = df[cols[7]].values

    ref_alpha_arr = np.zeros(lott_type_arr.shape)
    lott_alpha_arr = np.zeros(lott_type_arr.shape)
    beta_arr = np.zeros(lott_type_arr.shape)
    sFactor_arr = np.ones(lott_type_arr.shape)


    # Unpack params
    if len(params) == 2:
       # two scaling factors
       (cPlus_sFactor, cMinus_sFactor) = params
       
       # unpack params for trial in question
       sFactor_arr[lott_type_arr == cPlus_id] = cPlus_sFactor
       sFactor_arr[lott_type_arr == cMinus_id] = cMinus_sFactor

    elif len(params) == 4:
       # Two betas and two scaling factors
       (mt_cPlus_beta, mt_cMinus_beta,
        cPlus_sFactor, cMinus_sFactor) = params

       # unpack params for trial in question
       sFactor_arr[lott_type_arr == cPlus_id] = cPlus_sFactor
       sFactor_arr[lott_type_arr == cMinus_id] = cMinus_sFactor

       beta_arr[lott_type_arr == cMinus_id] = mt_cPlus_beta
       beta_arr[lott_type_arr == money_id] = mt_cMinus_beta

    negloglikelihood = _compute_nll(ref_prob_arr, ref_qt_arr, ref_alpha_arr,
                                   lott_prob_arr, lott_qt_arr, lott_alpha_arr,
                                   beta_arr, sFactor_arr, 
                                   choice_arr)

    return negloglikelihood

def _get_nll(params, df):
    '''
    Computes negative logLikelihood

    columns should have the following order:
              ['trial_type',
              'ref_type', 'ref_qt', 'ref_prob',
              'lott_type', 'lott_qt', 'lott_prob',
              'choice',
              'ref_alpha_est', 'lott_alpha_est', 'beta_est',
              'sFactor_est']
              '''
    
    cols = df.columns
    trial_type_arr = df[cols[0]].values
    reff_type_arr = df[cols[1]].values
    ref_prob_arr = df[cols[3]].values
    ref_qt_arr = df[cols[2]].values
    lott_type_arr = df[cols[4]].values
    lott_prob_arr = df[cols[6]].values
    lott_qt_arr = df[cols[5]].values
    choice_arr = df[cols[7]].values

    ref_alpha_arr = np.zeros(lott_type_arr.shape)
    lott_alpha_arr = np.zeros(lott_type_arr.shape)
    beta_arr = np.zeros(lott_type_arr.shape)
    sFactor_arr = np.ones(lott_type_arr.shape)

    # Unpack params
    if len(params) == 6:
       # Three alphas, one beta and two scaling factors (beta is unpacked directly)
       (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
       beta, 
       cPlus_sFactor, cMinus_sFactor) = params
       
       # unpack params for trial in question
       ref_alpha_arr[reff_type_arr == money_id] = st_money_alpha
       ref_alpha_arr[reff_type_arr == cPlus_id] = st_cPlus_alpha
       ref_alpha_arr[reff_type_arr == cMinus_id] = st_cMinus_alpha

       lott_alpha_arr[lott_type_arr == money_id] = st_money_alpha
       lott_alpha_arr[lott_type_arr == cPlus_id] = st_cPlus_alpha
       lott_alpha_arr[lott_type_arr == cMinus_id] = st_cMinus_alpha

       beta_arr[:] = beta

       sFactor_arr[lott_type_arr == cPlus_id] = cPlus_sFactor
       sFactor_arr[lott_type_arr == cMinus_id] = cMinus_sFactor

    elif len(params) == 6:
       # Three alphas, three betas 
       (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
       st_money_beta, st_cPlus_beta, st_cMinus_beta) = params

       # unpack params for trial in question
       ref_alpha_arr[reff_type_arr == money_id] = st_money_alpha
       ref_alpha_arr[reff_type_arr == cPlus_id] = st_cPlus_alpha      # This should never happen
       ref_alpha_arr[reff_type_arr == cMinus_id] = st_cMinus_alpha    # This should never happen

       lott_alpha_arr[lott_type_arr == money_id] = st_money_alpha
       lott_alpha_arr[lott_type_arr == cPlus_id] = st_cPlus_alpha
       lott_alpha_arr[lott_type_arr == cMinus_id] = st_cMinus_alpha
       
       beta_arr[(trial_type_arr == st_id) & (lott_type_arr == money_id)] = st_money_beta
       beta_arr[(trial_type_arr == st_id) & (lott_type_arr == cPlus_id)] = st_cPlus_beta
       beta_arr[(trial_type_arr == st_id) & (lott_type_arr == cMinus_id)] = st_cMinus_beta
       beta_arr[(trial_type_arr == mt_id) & (lott_type_arr == cMinus_id)] = mt_cPlus_beta
       beta_arr[(trial_type_arr == mt_id) & (lott_type_arr == money_id)] = mt_cMinus_beta

       sFactor_arr[lott_type_arr == cPlus_id] = cPlus_sFactor
       sFactor_arr[lott_type_arr == cMinus_id] = cPlus_sFactor


    negloglikelihood = _compute_nll(ref_prob_arr, ref_qt_arr, ref_alpha_arr,
                                   lott_prob_arr, lott_qt_arr, lott_alpha_arr,
                                   beta_arr, sFactor_arr, 
                                   choice_arr)

    return negloglikelihood

def stepwise_estimate_MultiOpt(args, x0, N_optimizers):

    def _get_iter_params(xk):
        iter_params_list.append(xk.tolist())
    
    df = args
    st_mask = df[optimize_cols[0]] == 'same'
    mt_mask = df[optimize_cols[0]] == 'mixed'

    cPlus_mask = df[optimize_cols[4]] == 'CS+'
    cMinus_mask = df[optimize_cols[4]] == 'CS-'

    # Unpack initialization parameters
    if len(x0) == 6:
        (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
            beta,
            cPlus_sFactor, cMinus_sFactor) = x0
        st_params = (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
                    beta)
        mt_params = (cPlus_sFactor, cMinus_sFactor)
        st_params_colNames = ['Money alpha', 'CS+ alpha', 'CS- alpha', 
                                'beta']
        mt_params_colNames = ['CS+ sFactor', 'CS- sFactor']
    elif len(x0) == 10:
        (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
            st_money_beta, st_cPlus_beta, st_cMinus_beta,
            mt_cPlus_beta, mt_cMinus_beta,
            cPlus_sFactor, cMinus_sFactor) = x0
        st_params = (st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, 
                    st_money_beta, st_cPlus_beta, st_cMinus_beta)
        mt_params = (mt_cPlus_beta, mt_cMinus_beta,
                    cPlus_sFactor, cMinus_sFactor)
        st_params_colNames = ['Money alpha', 'CS+ alpha', 'CS- alpha', 
                                'Money beta', 'CS+ st beta', 'CS- st beta', ]
        mt_params_colNames = ['CS+ mt beta', 'CS- mt beta',
                                'CS+ sFactor', 'CS- sFactor']

    for n_opt in range(N_optimizers):
        x0_st_params = tuple([random.uniform(*pars) for pars in st_params])
        # Estimate parameters from Same type trials
        iter_params_list = []
        res_st_ = minimize(_get_st_nll, x0_st_params, args=df.loc[st_mask,:],
                            callback=_get_iter_params)
        if n_opt == 0:
            res_st = res_st_
        else:
            if res_st_.fun < res_st.fun:
                res_st = res_st_

    st_iterParams_df = pd.DataFrame(iter_params_list, columns=st_params_colNames)

    # map same type estimation results to required fields
    df.loc[mt_mask, optimize_cols[8]] = res_st.x[0]                 # Money alpha
    df.loc[mt_mask & cPlus_mask, optimize_cols[9]] = res_st.x[1]    # CS+ alpha
    df.loc[mt_mask & cMinus_mask, optimize_cols[9]] = res_st.x[2]   # CS- alpha
    if len(x0) == 6:
        df.loc[mt_mask, optimize_cols[10]] = res_st.x[3]              # beta
    for n_opt in range(N_optimizers):
        x0_mt_params = tuple([random.uniform(*pars) for pars in mt_params])
        # Estimate parameters from mixed type trials
        iter_params_list = []
        res_mt_ = minimize(_get_mt_nll, x0_mt_params, args=df.loc[mt_mask,:],
                            callback=_get_iter_params)
        if n_opt == 0:
            res_mt = res_mt_
        else:
            if res_mt_.fun < res_st.fun:
                res_mt = res_mt_
    mt_iterParams_df = pd.DataFrame(iter_params_list, columns=mt_params_colNames)

    return res_st, res_mt, st_iterParams_df, mt_iterParams_df

In [29]:
def _simNsubsfit_MultiOpt(allTrials_df, N_subs, x0, 
                                 st_money_alpha_arr, st_cPlus_alpha_arr, st_cMinus_alpha_arr,
                                 st_money_beta_arr, st_cPlus_beta_arr, st_cMinus_beta_arr,
                                 mt_cPlus_beta_arr, mt_cMinus_beta_arr, 
                                 cPlus_sFactor_arr, cMinus_sFactor_arr,
                                 N_optimizers = 50,
                                 plot_behaviour = True, output = 'long_flags'):

    # ---------------------------------------------------------------------------
    # Prelocate mememory and/or create variables for outputs
    
    if len(x0) == 6:
        st_param_size = 4
        mt_param_size = 2
    elif len(x0) == 10:
        st_param_size = 6
        mt_param_size = 4
    
    st_estPars = np.zeros((st_param_size, N_subs))
    mt_estPars = np.zeros((mt_param_size, N_subs))
    if 'flags' in output:
        st_flags = []
        mt_flags = []
    if 'short' not in output:
        st_hessians = np.zeros((st_param_size, st_param_size, N_subs))
        mt_hessians = np.zeros((mt_param_size, mt_param_size, N_subs))
    if 'long' in output:
        st_iterParams_df = pd.DataFrame()
        mt_iterParams_df = pd.DataFrame()
    
    if plot_behaviour:
        subject_choiceCount_df = pd.DataFrame()
    
    for i in tqdm(range(N_subs)):
        # Get parameters per participant
        # ¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨
        st_money_alpha = st_money_alpha_arr[i]
        st_cPlus_alpha = st_cPlus_alpha_arr[i]
        st_cMinus_alpha = st_cMinus_alpha_arr[i]
        st_money_beta = st_money_beta_arr[i]
        st_cPlus_beta = st_cPlus_beta_arr[i]
        st_cMinus_beta = st_cMinus_beta_arr[i]
        mt_cPlus_beta = mt_cPlus_beta_arr[i]
        mt_cMinus_beta = mt_cMinus_beta_arr[i]
        cPlus_sFactor = cPlus_sFactor_arr[i]
        cMinus_sFactor = cMinus_sFactor_arr[i]
        
        # simulate subject behaviour
        # ¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨
        subjectTrials_df = sim._pack_subjectParameters(st_money_alpha, st_cPlus_alpha, st_cMinus_alpha, st_money_beta, 
                                              st_cPlus_beta, st_cMinus_beta, mt_cPlus_beta, mt_cMinus_beta, 
                                              cPlus_sFactor, cMinus_sFactor, 
                                              allTrials_df)

        subjectTrials_df.insert(7, optimize_cols[7], _get_choices(subjectTrials_df))

        # Pack initial estimates and fit model
        # ¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨
        res_st, res_mt, _st_iterParams_df, _mt_iterParams_df = stepwise_estimate_MultiOpt(subjectTrials_df, x0, N_optimizers)
        
        # prepare plot
        # ¨¨¨¨¨¨¨¨¨¨¨¨
        if plot_behaviour:
            # Compute Probability of chossing lottery per subject
            _subject_choiceCount_df = pd.DataFrame(subjectTrials_df[beahviour_cols].groupby(
                            list(subjectTrials_df[beahviour_cols].columns[:-1])
                            ).apply(
                        lambda df: necon.get_probLottery(df)), 
                        columns = [_v_.probLotteryChoice_colName]).reset_index()
            _subject_choiceCount_df['n_sub'] = i

            subject_choiceCount_df = pd.concat([subject_choiceCount_df, _subject_choiceCount_df], axis = 0)

        # Prepare OUTPUTS
        # ¨¨¨¨¨¨¨¨¨¨¨¨¨¨¨
        st_estPars[:, i] = res_st.x
        mt_estPars[:, i] = res_mt.x

        if 'flags' in output:
            st_flags.append(res_st.message)
            mt_flags.append(res_mt.message)

        if 'short' not in output:
            st_hessians[:, :, i] = res_st.hess_inv
            mt_hessians[:, :, i] = res_mt.hess_inv

        if 'long' in output:
            _st_iterParams_df = _st_iterParams_df.reset_index().rename(columns={'index':'iter'},)
            _st_iterParams_df['n_sub'] = i
            
            _mt_iterParams_df = _mt_iterParams_df.reset_index().rename(columns={'index':'iter'},)
            _mt_iterParams_df['n_sub'] = i

            st_iterParams_df = pd.concat([st_iterParams_df, _st_iterParams_df], axis = 0)
            mt_iterParams_df = pd.concat([mt_iterParams_df, _mt_iterParams_df], axis = 0)
    # ---------------------------------------------------------------------------
    # Pack outputs
    if output == 'short':
        out_vars = (st_estPars, mt_estPars)
    elif output == 'short_flags':
        out_vars = (st_estPars, mt_estPars, st_flags, mt_flags)
    elif output == 'medium':
        out_vars = (st_estPars, mt_estPars, st_hessians, mt_hessians)
    elif output == 'medium_flags':
        out_vars = (st_estPars, mt_estPars, st_hessians, mt_hessians, st_flags, mt_flags)
    elif output == 'long':
        out_vars = (st_estPars, mt_estPars, st_hessians, mt_hessians, st_iterParams_df, mt_iterParams_df)
    elif output == 'long_flags':
        out_vars = (st_estPars, mt_estPars, st_hessians, mt_hessians, st_iterParams_df, mt_iterParams_df, st_flags, mt_flags)

    if plot_behaviour:
        out_vars += (subject_choiceCount_df,)
    return out_vars

def simANDfit_multiParticipants(allTrials_df, N_subs, x0, 
                                 mean_std_st_money_alpha, mean_std_st_cPlus_alpha, mean_std_st_cMinus_alpha,
                                 mean_std_st_money_beta, mean_std_st_cPlus_beta, mean_std_st_cMinus_beta,
                                 mean_std_mt_cPlus_beta, mean_std_mt_cMinus_beta, 
                                 mean_std_cPlus_sFactor, mean_std_cMinus_sFactor,
                                 N_optimizers = 20,
                                 plot_behaviour = True, startFromBehaviour = False,
                                 output = 'long_flags'):

    # ---------------------------------------------------------------------------
    # Create array of parameters used for behaviour simulation

    st_money_alpha_arr = abs(np.random.normal(*mean_std_st_money_alpha, N_subs))
    st_cPlus_alpha_arr = abs(np.random.normal(*mean_std_st_cPlus_alpha, N_subs))
    st_cMinus_alpha_arr = abs(np.random.normal(*mean_std_st_cMinus_alpha, N_subs))
    st_money_beta_arr = abs(np.random.normal(*mean_std_st_money_beta, N_subs))

    cPlus_sFactor_arr = abs(np.random.normal(*mean_std_cPlus_sFactor, N_subs))
    cMinus_sFactor_arr = abs(np.random.normal(*mean_std_cMinus_sFactor, N_subs))

    if len(x0) == 6:
        #model_type = '3 alphas, 1 beta and 2 sFactors'
        st_cPlus_beta_arr = st_money_beta_arr
        st_cMinus_beta_arr = st_money_beta_arr
        mt_cPlus_beta_arr = st_money_beta_arr
        mt_cMinus_beta_arr = st_money_beta_arr
        
        # pack parameters
        st_pars = np.stack([st_money_alpha_arr, st_cPlus_alpha_arr, st_cMinus_alpha_arr,
                        st_money_beta_arr,] )
        mt_pars = np.stack([cPlus_sFactor_arr, cMinus_sFactor_arr])

    else:
        # model_type = '3 alphas, 5 beta and 2 sFactors'    
        st_cPlus_beta_arr = abs(np.random.normal(*mean_std_st_cPlus_beta, N_subs))
        st_cMinus_beta_arr = abs(np.random.normal(*mean_std_st_cMinus_beta, N_subs))
        mt_cPlus_beta_arr = abs(np.random.normal(*mean_std_mt_cPlus_beta, N_subs))
        mt_cMinus_beta_arr = abs(np.random.normal(*mean_std_mt_cMinus_beta, N_subs))
        # pack parameters
        st_pars = np.stack([st_money_alpha_arr, st_cPlus_alpha_arr, st_cMinus_alpha_arr,
                        st_money_beta_arr, st_cPlus_beta_arr, st_cMinus_beta_arr,
                        ] )
        mt_pars = np.stack([mt_cPlus_beta_arr, mt_cMinus_beta_arr,
                            cPlus_sFactor_arr, cMinus_sFactor_arr])
    
    # ---------------------------------------------------------------------------
    # Start Loop
    warnings.filterwarnings("ignore")#, category=RuntimeWarning)       # Ignore Optimization Warnings

    try:
        len(x0[0])>1
        # Run multiple optimizers (minRange, maxRange, N_optimizers)
        print('Running {} optimizers per subject with random initial estimates'.format(N_optimizers))
        out_vars = _simNsubsfit_MultiOpt(allTrials_df, N_subs, x0, 
                                 st_money_alpha_arr, st_cPlus_alpha_arr, st_cMinus_alpha_arr,
                                 st_money_beta_arr, st_cPlus_beta_arr, st_cMinus_beta_arr,
                                 mt_cPlus_beta_arr, mt_cMinus_beta_arr, 
                                 cPlus_sFactor_arr, cMinus_sFactor_arr,
                                 N_optimizers = N_optimizers,
                                 plot_behaviour = True, output = 'long_flags')
    except TypeError:
        if startFromBehaviour:
            x0_arr = np.concatenate([st_pars, mt_pars])
            print('Starting optimization from true parameters')
        else:
            print('Starting optimization with fixed values as initial estimates')
            x0_arr = np.repeat(np.expand_dims(np.array(x0), 1), N_subs, axis =1)

        # Fit model with one optimizer
        out_vars =sim._simNsubsfit_oneOptimizer(allTrials_df, N_subs, x0_arr, 
                                 st_money_alpha_arr, st_cPlus_alpha_arr, st_cMinus_alpha_arr,
                                 st_money_beta_arr, st_cPlus_beta_arr, st_cMinus_beta_arr,
                                 mt_cPlus_beta_arr, mt_cMinus_beta_arr, 
                                 cPlus_sFactor_arr, cMinus_sFactor_arr,
                                 plot_behaviour = True, output = 'long_flags')
        
    out_vars = (st_pars, mt_pars, ) + out_vars
    
    # ---------------------------------------------------------------------------
    # Plot Behaviour if requested
    if plot_behaviour:
        subject_choiceCount_df = out_vars[-1]
        out_vars = out_vars[:-1]
        sns.set(font_scale=1.5)
        # CREATE FUNTION IN PLOTS FOR THIS!!
        # Throwing a depreciation warning that I want to filter
        g = sns.relplot(
            data=subject_choiceCount_df.reset_index(drop=True), x=beahviour_cols[-3], y=_v_.probLotteryChoice_colName, 
            col=beahviour_cols[-4], row = beahviour_cols[0],
            hue=beahviour_cols[-2], style=beahviour_cols[-2], kind="line",facet_kws={'sharex': False, 'margin_titles' : True},
        )
        g.fig.suptitle('Behaviour across simulated subjects\nmean and 95%CI (N={})'.format(N_subs), va='bottom');
    
    warnings.filterwarnings("always")#, category=RuntimeWarning)        # Turn Warnings back on
    
    # ---------------------------------------------------------------------------

    return out_vars

# Vectorized computation

# Fixed Task Parameters

In [30]:
uniqueLott_Nreps= 6      # Unique Lottery Repititions  

# Same-type & mixed type Trials Lottery probabilities
st_refPs = [1]                              # Reference option
st_lottPs = [0.13, 0.22, 0.38, .50, .75]    # Lottery option

# Same-type task variables
st_money_refQs = [1]                               # Euros
st_money_lottQs = [1, 2, 5, 12, 20]                # Euros

st_cPlus_refQs = [20]                              # mL of CS+ yogurt 
st_cPlus_lottQs = [20, 40, 80, 120, 200]           # mL of CS+ yogurt

st_cMinus_refQs = st_cPlus_refQs                      # mL of CS- yogurt 
st_cMinus_lottQs = st_cPlus_lottQs                    # mL of CS- yogurt 

# Mixed-type task variables
mt_refQs = [.2]                                # Euros
mt_refPs = [1]
mt_lottPs = [0.13, 0.22, 0.38, .50, .75]

mt_cPlus_lottQs = [40, 80, 120, 150, 200]      # mL of CS+ yogurt 
mt_cMinus_lottQs = mt_cPlus_lottQs            # mL of CS- yogurt 


allTrials_df = sim.pack_taskParameters(
                st_refPs, st_lottPs, st_money_refQs, st_money_lottQs, st_cPlus_refQs, st_cPlus_lottQs,
                st_cMinus_refQs, st_cMinus_lottQs, mt_refQs, mt_refPs, mt_lottPs, mt_cPlus_lottQs,
                mt_cMinus_lottQs, uniqueLott_Nreps)

print('Trials per type:\n{}'.format(allTrials_df['trial_type'].value_counts()))

Trials per type:
same     450
mixed    300
Name: trial_type, dtype: int64


### Initialization Parameters

In [31]:
alphaMoney0 = (0.1, 1.5)
alphaCplus0 = (0.1, 1.5)
alphaCminus0 = (0.1, 1.5)

st_betaMoney0 = (0.5, 15) # also used in model with only one beta
st_betaCplus0 = (0.5, 15)
st_betaCminus0 = (0.5, 15)
mt_betaCplus0 = (0.5, 15)
mt_betaCminus0 = (0.5, 15)

sFactorCplus0 = (0.01, 0.5)
sFactorCminus0= (0.01, 0.5)


x0_10params = (alphaMoney0, alphaCplus0, alphaCminus0, 
      st_betaMoney0, st_betaCplus0, st_betaCminus0, mt_betaCplus0, mt_betaCminus0,
      sFactorCplus0, sFactorCminus0)

x0_6params = (alphaMoney0, alphaCplus0, alphaCminus0, 
      st_betaMoney0,
      sFactorCplus0, sFactorCminus0)

In [25]:
# INPUTS
allTrials_df = allTrials_df.copy()
N_subs = 20
x0 = x0_10params
#                         (mean, sd)
mean_std_st_money_alpha = (0.7, 0.1)
mean_std_st_cPlus_alpha = (0.8, 0.1)
mean_std_st_cMinus_alpha = (0.8, 0.1)
mean_std_st_money_beta = (1.8, 0.3)
mean_std_st_cPlus_beta = mean_std_st_money_beta
mean_std_st_cMinus_beta = mean_std_st_money_beta
mean_std_mt_cPlus_beta = (10, 0.3)
mean_std_mt_cMinus_beta = mean_std_st_money_beta
mean_std_cPlus_sFactor = (.02, 0.005)
mean_std_cMinus_sFactor = mean_std_cPlus_sFactor

plot_behaviour = True
output = 'long_flags'

(st_pars, mt_pars, 
 st_estPars, mt_estPars, 
 st_hessians, mt_hessians, 
 st_iterParams_df, mt_iterParams_df, 
 st_flags, mt_flags) = simANDfit_multiParticipants(allTrials_df, N_subs, x0, 
                                    mean_std_st_money_alpha, mean_std_st_cPlus_alpha, mean_std_st_cMinus_alpha,
                                    mean_std_st_money_beta, mean_std_st_cPlus_beta, mean_std_st_cMinus_beta,
                                    mean_std_mt_cPlus_beta, mean_std_mt_cMinus_beta, 
                                    mean_std_cPlus_sFactor, mean_std_cMinus_sFactor,
                                    plot_behaviour = True, startFromBehaviour = True,
                                    N_optimizers=10, output = 'long_flags')

Running 10 optimizers per subject with random initial estimates


  0%|          | 0/20 [00:00<?, ?it/s]


IndexError: index 0 is out of bounds for axis 0 with size 0